In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import fitsio
from astropy.io import fits
import pandas as pd 
import sys
from quickbin import bin2d
from pyarrow import parquet

In [3]:
def centile_clip(image, centiles=(1, 99.5)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)
    
    if isinstance(image, np.ma.MaskedArray):
        
        return result
    
    return result.data

In [4]:
eclipse = 'e23456'

image = fits.open(f"/home/bekah/gPhoton2/test_data/{eclipse}/{eclipse}-nd-ffull-b00-image-r.fits")
movie = fits.open(f"/home/bekah/gPhoton2/test_data/{eclipse}/{eclipse}-nd-f0120-b00-movie-r.fits") 

In [30]:
image[1].header

XTENSION= 'IMAGE   '                                                            
BITPIX  =                  -32 / data type of original image                    
NAXIS   =                    2 / dimension of original image                    
NAXIS1  =                 3065 / length of original image axis                  
NAXIS2  =                 3099 / length of original image axis                  
PCOUNT  =                    0                                                  
GCOUNT  =                    1                                                  
EXTNAME = 'CNT     '                                                            
CDELT1  = -0.000416666666666667                                                 
CDELT2  = 0.000416666666666667                                                  
CTYPE1  = 'RA---TAN'                                                            
CTYPE2  = 'DEC--TAN'                                                            
CRPIX1  =                153

In [31]:
movie[1].header

XTENSION= 'IMAGE   '                                                            
BITPIX  =                  -32 / data type of original image                    
NAXIS   =                    3 / dimension of original image                    
NAXIS1  =                 3065 / length of original image axis                  
NAXIS2  =                 3099 / length of original image axis                  
NAXIS3  =                   15 / length of original image axis                  
PCOUNT  =                    0                                                  
GCOUNT  =                    1                                                  
EXTNAME = 'CNT     '                                                            
CDELT1  = -0.000416666666666667                                                 
CDELT2  = 0.000416666666666667                                                  
CTYPE1  = 'RA---TAN'                                                            
CTYPE2  = 'DEC--TAN'        

In [27]:
# cnt 

combined_array_cnt = np.sum(movie[1].data, axis=0)
move_image_sub = image[1].data - combined_array_cnt
hdu2 = fits.PrimaryHDU(move_image_sub)
hdulist = fits.HDUList([hdu2])
hdulist.writeto(f'e{eclipse}_cntdiff', overwrite=True)
print(f"done w {eclipse}")

done w e23456


/tmp/ipykernel_32310/3587008740.py:4: RuntimeWarning: invalid value encountered in subtract
  move_image_sub = image[1].data - combined_array_cnt


In [9]:
movie[2].data[14]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [10]:
# flags / mask

combined_array_mask = np.sum(movie[2].data, axis=0)-movie[2].data[14]
move_image_sub = image[2].data - combined_array_mask.astype(np.float64)
hdu2 = fits.PrimaryHDU(move_image_sub)
hdulist = fits.HDUList([hdu2])
hdulist.writeto(f'e{eclipse}_flagdiff', overwrite=True)



In [29]:
# edge

combined_array_edge = np.sum(movie[3].data, axis=0)
move_image_sub = image[3].data - combined_array_edge.astype(np.float64)
hdu2 = fits.PrimaryHDU(move_image_sub)
hdulist = fits.HDUList([hdu2])
hdulist.writeto(f'e{eclipse}_edgediff', overwrite=True)
